In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool

# download data from browser
loader = WebBaseLoader("https://neurons-lab.com/")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100
).split_documents(docs)


In [ ]:
# option!! use huggungface embedding

from langchain.embeddings import HuggingFaceEmbeddings

#param encode_kwargs: Dict[str, Any] [Optional]
#Keyword arguments to pass when calling the encode method of the Sentence Transformer model, such as prompt_name, prompt, batch_size, precision, normalize_embeddings, and more. See also the Sentence Transformer documentation: https://sbert.net/docs/package_reference/SentenceTransformer.html#sentence_transformers.SentenceTransformer.encode

#param model_kwargs: Dict[str, Any] [Optional]¶
#Keyword arguments to pass to the Sentence Transformer model, such as device, prompts, default_prompt_name, revision, trust_remote_code, or token. See also the Sentence Transformer documentation: https://sbert.net/docs/package_reference/SentenceTransformer.html#sentence_transformers.SentenceTransformer

#param multi_process: bool = False
#Run encode() on multiple GPUs.

#param show_progress: bool = False
#Whether to show a progress bar.

model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs)

In [4]:
# this will open a folder named aoai_chroma_db2 and create a sqlite file 

persist_directory = 'aoai_chroma_db2'
vectordb = Chroma.from_documents(documents=documents, embedding=embedding, persist_directory=persist_directory)
vectordb.persist()


retriever = vectordb.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    "neurons_lab_search",
    "Search for information about Neurons Lab. For any questions about Neurons Lab, you must use this tool!",
)

NameError: name 'Chroma' is not defined

In [ ]:
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults
from langchain.agents import initialize_agent, AgentType
# from langchain.chat_models import ChatOpenAI

from langchain.chat_models import AzureChatOpenAI

# # Configure Azure OpenAI Service API
# openai.api_type = "azure"
# openai.api_version = "2024-02-15-preview"
# openai.api_base = "https://azure-openai-test-s100.openai.azure.com/"
# openai.api_key = "6cb5bfda3d7241dfb35c518668a4c099"

client = AzureChatOpenAI(
    deployment_name="gpt-35-turbo-16k",
    temperature=0,
    api_key="6cb5bfda3d7241dfb35c518668a4c099",
    azure_endpoint="https://azure-openai-test-s100.openai.azure.com/",
    openai_api_version="2024-02-15-preview")


agent_chain = initialize_agent(
    [retriever_tool], #, tavily_tool],
    llm=client,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [ ]:
# run the agent
agent_chain.run(
    "What are the Neurons Lab services",
)

In [1]:
# add yourself tools (defined function)

#use the @tooldecorator to make Langchain know about it
#describe function comment between """ """

from langchain.tools import BaseTool, StructuredTool, tool

@tool
def calculate_length_tool(a: str) -> int:
    """The function calculates the length of the input string."""
    return len(a)


/Users/lizylin/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
from langchain.agents import initialize_agent

agent_chain2 = initialize_agent(
    [retriever_tool, calculate_length_tool],
    llm=client,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

NameError: name 'retriever_tool' is not defined